In [11]:
import re
import os
import sqlparse

In [12]:
# control_path = r"C:\Merwin\BTEQWorkFiles\BTEQ\Converted\Enwisen\HDW_Candidate_Onboarding_Resource_WRK.sql"
control_path = r"C:\Merwin\BTEQWorkFiles\BTEQ\Converted\Enwisen"
substr = '`hca-hin-dev-cur-hr`.'
db_objects = []
db_schema = 'edwhr'
path = r"OutputFiles\tables.csv"

In [13]:
def read_file(file_path):
    file1 = open(file_path, 'r')
    return file1.read()

In [14]:
def Filter(string, substr):
    return [str.removeprefix(substr).replace('_copy','') for str in string if
            #  any(sub in str for sub in substr)
            substr in str
             ]

In [15]:
def append_db_objects(db_object):
    if db_object not in db_objects :
        db_objects.append(db_object)

In [16]:
def identify_objects(Lines):
    # Strips the newline character
    for line in Lines:
        words = line.split()
        matches = Filter(words, substr)
        # matches[0].replace(";","")
        if matches :
            db_object_words = matches[0].split('.')
            db_object_words[1] = db_object_words[1].replace(";","")
            append_db_objects(",".join(db_object_words))
            if db_object_words[0] == db_schema:
                append_db_objects(",".join([db_schema + "_base_views", db_object_words[1]]))
            if db_object_words[0] == db_schema + "_base_views":
                append_db_objects(",".join([db_schema, db_object_words[1]]))


In [17]:
if os.path.isdir(control_path):
    for file in os.listdir(control_path):
        if file.endswith(".sql"):
            file_path = f"{control_path}\{file}"
            Lines = sqlparse.format(read_file(file_path),strip_comments=True).splitlines()
            identify_objects(Lines)
else:
    Lines = sqlparse.format(read_file(control_path),strip_comments=True).splitlines()
    identify_objects(Lines)


In [18]:
db_objects.sort()

In [19]:
with open(path, 'w') as file:
    file.writelines("Database,Table\n")
    file_string = '\n'.join(db_objects)
    file.write(file_string)